# ECM2423 - Artificial Intelligence and Applications
## Coursework exercise (This notebook contains Q2-Q2.2)
### Student No: 700074704
***

### Question 2.1: Design and implement the Sudoku problem using Evolutionary algorithm.
***
<h4>2.1.1. In first problem design your evolutionary algorithm addressing the following points in your design process you should first briefly outline the how you representing Sudoko problem </h4>

***
(a) Choose an appropriate solution space and solution representation

I have defined the sudoku puzzle as a one-dimensional chromosome with a total length of 81 integer numbers (In this case I have chosen the traditional 9 x 9 puzzle). Each block (3 x 3) of the puzzle also consists of linked sub-chromosomes. The solution space consists of the total set of solutions to the puzzle, i.e the solution space will be the 9x9 grid in which all the squares are filled. 
I will try to find the optimal solution using a genetic algorithm. In this case a solution consists of the given values in each chromosome, and the generated values such that each row is a valid permutation of the values 1 through 9. For this problem, I have made use of a 2D list to model the sudoku grid, wherein each grid contains 9 sub-lists.

(b) Define an appropriate fitness function

Our aim for the Sudoku problem is to have the sum of each row and each column equal to 45 without any repeats in the same column, row or 3x3 grid. So for the fitness function I have chosen to sum the number of repeats in the entire grid. In this case, a lower number would represent a better fit. 
Therefore, for example, a fitness of 0 is ideal as indicates that the Sudoku puzzle does not contain any duplicates in the same row or column.

(c) Define a crossover operator for the chosen representation.

Every sub-block is viewed as an individual problem, and uniform crossover can be applied to a specific block, in which the crossover positions are restricted to being between only the sub-blocks. 
For this algorithm I take into account two parents and slice them to produce a new child individual. This slice location is chosen by random. For example, if the slice was between row 2 and row 3, the first parent would take the first two rows of the child, and the second parent would take the third to the ninth rows of the child. 

(d) Define a mutation operator for the chosen representation

Swap mutation technique is used so that it avoids the duplication of numerals within a sub-block. 
For this algorithm, I make use of a mutation rate modifier which randomly decides if a row should be mutated or not.
Because of this the maximum a row could be mutated is nine times (two numbers swapped every row).

(e) Decide how to initialise the population

The population of solutions is initialised by first inputting the base sudoku grid and filling in the pre-given squares, after this I generate random individuals (with numerals 1-9) that are not already present in the row for any of the remaining subgrid spaces. This method ensures that there are no duplicates in a row at the time of intilisation and is much more effective than simply generating random numbers.

(f) Decide selection and replacement methods.

For the selection method I have chosen to do a tournament selection, that is, randomly selected individuals are compared, and the one which has the better fitness (by using the fitness function) is kept for crossover. The number of these selected individuals is defined by the 'truncation rate'.

The replacement method ensures that the best performing parents are crossed over to the next generation, and the number of these parents to be carried over is defined by the 'replacement rate'.

(g) Choose an appropriate termination criterion

There are two ways to terimate the algorithm. The first being that once the maximum fitness has been achieved (that is fitness function of the best individual in an child population is 0), which would indicate that a correct soluation has been found and the algorithm terminates. 

The second is to deal with situations wherein it is impractical for a solution to be reached. In these sorts of situations, the user can also specify a limit for the number of maximum generations ('NUMBER_GENERATION') the algorithm should run through before it stops. This is effective in case, for example, the algorithm is stuck in the local minima for a long time. 

I have also impleted a method in place to prevent being stuck in the local minima. If it is found that the fitness function remains constant over multiple generations (which would indicate a local minima), I significantly increase the mutation rate for a while to try to escape this local minima. Due to this, the algorithm may end up giving a poor fitness than which it had before. Because of this I keep track of the best solution so far and use this as the final result.


***

 <h4> 2.1.2. Then, you should implement the evolutionary algorithm in Python to solve the Sudoku problem.  </h4>

***

In [12]:
# (EA) Genetic Algorithm to solve the Sudoku problem 
# Please refer to GA_Sudoku.md to find details on how to run this program.
# I have also attached this python script separately as GA_Sudoku.py for ease of testing.

# Import the following necessary packages to run the program 
from random import random, randint, choice, sample, shuffle
from time import time
from copy import deepcopy
import sys

#create an array to hold numbers for the base grid
grid = []
#create an array to hold numbers in the grid that may be changed
temp = []

def make_grid(grid_file):
    """
    Method to read the given base grid file and develop the grid from it.
    Keyword arguments:
        grid_file -- the base grid file to be read from
    """ 
    with open(grid_file, "r") as base_file:
        grid_input = base_file.readlines()
        
        for line in grid_input:
            row_present = []
            row_temp = []
            position = 0
            
            # makes sure to ignore the divider row, '!', and '.' from the base grid file
            if line[0] == '-':
                continue
                
            for value in line:
                #if the value does not have a pre-defined number in its position, replace it with a '0' and remember its position.
                if value == '.':
                    row_present.append(0)
                    row_temp.append(position)
                    position += 1
                    
                elif value != '\n' and value != "!":
                # Add the pre-defined numbers to the grid
                    row_present.append(int(value))
                    position += 1
                
            grid.append(row_present)
            temp.append(row_temp)
        
def evolve():
    """
    function which performs the evolution with help of calling on other operators.
    """
    population = create_pop()
    fitness_pop = evaluate_pop(population)
    
    counter = 0
    best_score = [[],100, 0]
    best = 100
    
    for gen in range(NUMBER_GENERATION):
        mating_pool = select_pop(population, fitness_pop)
        offspring_population = crossover_pop(mating_pool)
        
        #if stuck in local minima, try mutation escape.
        if SUPER_MUTATE and counter > SUPER_MUTATE:
            #if user specified verbose print the following string to the screen
            if VERBOSE:
                print("Stuck in local minima, trying super mutuation escape...")
            mutate_pop(offspring_population, 1)
            
        else:
            mutate_pop(offspring_population, MUTATION_RATE)
            population = offspring_population
            
            fitness_pop = evaluate_pop(population)
            best_ind, best_fit = best_pop(population, fitness_pop)
            
            # remember the best score
            if best_fit < best_score[1]:
                best_score[0], best_score[1], best_score[2] = best_ind, best_fit, gen
                
            if VERBOSE:
                #change this to mock code laterrrrr
                #print to screen if user specified verbose
                print("Generation #", gen, "Best fit: ", best_fit)
                
            #terminate if solution is found
            if best_fit in range(best-TOLERANCE, best+TOLERANCE+1):
                counter += 1
            else:
                best = best_fit
                counter = 0
            if TIME_MAX and time() - START > TIME_MAX:
                break
                
        total_time = time() - START
        
        #print the results to the command line
        
        print ('-'*40)
        print("\n Result made from base grid: ", GRID_FILE)
        print("Population size: ", POPULATION_SIZE)
        print("With Max Generations: ", NUMBER_GENERATION)
        print("\n Best Fitness: ", best_score[1])
        print("Generation : ", best_score[2])
        print("Solution found in ", total_time, "seconds")
        print("\n Result grid:\n")
        print_grid(best_score[0])
        print ('-'*40)    

### POPULATION-LEVEL OPERATORS ###

def create_pop():
    """
    Method to create the inital population and return a list of the individuals in the population
    """
    return [create_ind() for _ in range(POPULATION_SIZE)]          
    
            
def evaluate_pop(population):
    """
    Function that orders the population starting with the best fitness. 
    It returns a list of tuples wherein each tuple contains both the individual and its corresponding fitness value.
    
    Keyword arguments:
        
    population -- the population to be evaluated.
    """
    return [evaluate_ind(individual) for individual in population]
            

def select_pop(population, fitness_pop):
    """
    Method to take a percentage (which is input using constants) of the best individuals.
    It returns the new selected population. 
    
    Keyword arguments:
    
    population --  the population to be selected
    
    """
    #sorts the population
    sorted_population = sorted(zip(population, fitness_pop), key = lambda ind_fit: ind_fit[1])
    #takes a percentage of the best individuals
    return [ individual for individual, fitness in sorted_population[:int(POPULATION_SIZE * TRUNCATION_RATE)] ]

def crossover_pop(population):
    """
    Method to take the best parents, and then add to the children
    It returns the children population.
    """
    children = population[:REPLACE_NO]
    #randomly cross individuals to make up the remaining children population
    for _ in range(POPULATION_SIZE - REPLACE_NO):
            children.append(crossover_ind(choice(population), choice(population)))
    
    return children

def mutate_pop(population, mutation_rate):
    """
    Method to muttate the population according to the given rate. 
    
    Keyword arguments:
    
    population --  the population to be evaluated
    mutation_rate -- the rate of mutation (from 0-1).
    
    """
    for individual in population:
        mutate_ind(individual, mutation_rate)

def best_pop(population, fitness_pop):
    """
    Method to find the best individual in a given population. 
    It returns the individual with the best fitness.
    
    Keyword arguments:
    
    population --  the population to be evaluated
    fitness_pop -- the list which contains the fitness of the given population to be evaluated
    
    """           
    return sorted(zip(population, fitness_pop), key = lambda ind_fit: ind_fit[1])[0]

### INDIVIDUAL-LEVEL OPERATORS ###

def create_ind():
    """
    Function to create a new individual using the grid. 
    It returns a new individual.
    
    """  
    individual = deepcopy(grid)
    row_position = 0
            
    for this_row in individual:
            
            # Try to see which numbers are leftover and see if they can fill the row
            leftover_nums = {1, 2, 3, 4, 5, 6, 7, 8, 9} - set(this_row)
            leftover_nums = list(leftover_nums)
            
            # From the numbers which are left, shuffle randomly
            shuffle(leftover_nums)
            
            # Take these shuffled numbers add them to the non pre-determined row positions.
            for position in temp[row_position] :
                this_row[position] = leftover_nums.pop()
            
            assert len(leftover_nums) == 0
            row_position += 1
            
    return individual
    

def evaluate_ind(individual):
    """
    Get the fitness value for an individual. 
    It returns the fitness value of an individual.
    
    Keyword arguments:
    
    individual -- the individual to be evaluated.
    
    """ 
    # Count the number of repeats in a sub-chromosome.
    repeats = 0
    
    for i in range(0, 8, 3):
            repeats +=(9 - len(set(individual[i][:3] + individual[i+1][:3] + individual[i+2][:3])))
            repeats +=(9 - len(set(individual[i][3:6] + individual[i+1][3:6] + individual[i+2][3:6])))
            repeats +=(9 - len(set(individual[i][6:9] + individual[i+1][6:9] + individual[i+2][6:9])))
            
    # Count the number of repeats in a row
    for this_row in individual:
            repeats += (9- len(set(this_row)))
    # Count the number of repeats in a column.
    
    for column_position in range(9):
            column = []
            for this_row in individual:
                column.append(this_row[column_position])
            repeats += (9 - len(set(column)))
    
    # return the total number of repeats    
    return repeats
            
def crossover_ind(individual1, individual2):
    """
    Function to cross over two parents. 
    It returns the crossed child.
    
    Keyword arguments:
    
    individual1 -- the first individual parent to crossed over.
    individual2 -- the second individual parent to be crossed over.
    
    """ 
    crossover = randint(1, 8)
    return deepcopy(individual1[:crossover]) + deepcopy(individual2[crossover:])

def mutate_ind(individual, mutation_rate):
    """
    Function to mutate an individual. 
    It randomly (according to the set rate) swaps the places of 2 numbers in rows.
    This function returns the crossed child.
    
    Keyword arguments:
    
    individual -- refers to the individual to be mutated
    mutation_rate -- the rate of mutation (0-1) to be applied 
    
    """ 
    for x in range(9):
            
        if random() < mutation_rate and len(temp[x]) > 1:
            # get the positions of the two numbers to swap
            swap = sample(temp[x], 2)
            # make the swap
            individual[x][swap[0]], individual[x][swap[1]] = individual[x][swap[1]], individual[x][swap[0]]
            
#The following are functions to help with printing to the screen 
            
def print_list(list):
    """
    Function print items in the list to the screen. 

    Keyword arguments:
    list -- refers to be printed to the screen
    
    """ 
    for item in list:
            print(item, end='')
  
def print_grid(indv):
    """
    Method to format the grid to make it easy-to-read.
    It then prints the grid to the screen.
    
    Keyword arguments:
    individual -- refers to the grid to print.
    """
    for row_position in range(9):
    
        if row_position % 3 == 0 and row_position:
            print('-'*3+'|'+'-'*3+'|'+'-'*3)
              
        print_list(indv[row_position][:3])
        print('|', end='')
        print_list(indv[row_position][3:6])
        print('|', end='')
        print_list(indv[row_position][6:9])
        print('')
    print('\n')


### PARAMERS VALUES GIVEN BY INPUT ###

# Enter the file path to the base grid here...
GRID_FILE = '/Users/shreya/Desktop/Grid1.ss'

#Specify the maximum number of generations for the algorithm here
NUMBER_GENERATION = 1000

#Specify the size of the population
POPULATION_SIZE = 10

#Specify the number of individuals to be saved (in percentage)
TRUNCATION_RATE = 0.6

#Specify the number of parents to be replaced (in percentage)
REPLACEMENT_RATE = 0.2

#Use 'SUPER_MUTATE= 0' to disable super mutations, or specify the number of generations to be unchanged before performing super mutate.
SUPER_MUTATE = 50

#Specify the tolerance leiency 
TOLERANCE = 1

#Use 'TIME_MAX= 0' for no time limit, or apply specific time limit (in seconds).
TIME_MAX = 0

#specify 'verbose = 1' to see details of each generation printed to the command line            
VERBOSE = 1
      
make_grid(GRID_FILE)
MUTATION_RATE = 1/len(grid)

REPLACE_NO = int(POPULATION_SIZE * REPLACEMENT_RATE)

# Begin timer
START = time()
            
### EVOLVE! 
evolve()         


Generation # 0 Best fit:  44
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  44
Generation :  0
Solution found in  0.003016948699951172 seconds

 Result grid:

396|285|147
536|942|871
431|267|895
---|---|---
354|916|872
513|728|964
812|354|769
---|---|---
672|583|491
564|872|139
718|394|625


----------------------------------------
Generation # 1 Best fit:  43
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  43
Generation :  1
Solution found in  0.0071260929107666016 seconds

 Result grid:

326|815|947
986|742|351
421|567|893
---|---|---
459|816|732
723|618|954
812|934|756
---|---|---
392|658|471
563|872|149
574|318|629


----------------------------------------
Generation # 2 Best fit:  37
----------------------------------------

 Result made fro

372|516|489
569|873|124
428|319|657


----------------------------------------
Generation # 25 Best fit:  30
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  29
Generation :  20
Solution found in  0.4748961925506592 seconds

 Result grid:

328|915|647
586|742|391
231|467|895
---|---|---
754|916|832
963|178|524
812|594|763
---|---|---
372|516|489
569|873|124
428|319|657


----------------------------------------
Generation # 26 Best fit:  31
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  29
Generation :  20
Solution found in  0.4782440662384033 seconds

 Result grid:

328|915|647
586|742|391
231|467|895
---|---|---
754|916|832
963|178|524
812|594|763
---|---|---
372|516|489
569|873|124
428|319|657


----------------------------------------
Generatio

Generation # 44 Best fit:  23
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  23
Generation :  38
Solution found in  0.9098091125488281 seconds

 Result grid:

392|185|647
786|942|531
431|267|895
---|---|---
854|916|732
973|651|284
815|423|769
---|---|---
172|536|498
569|872|143
428|319|657


----------------------------------------
Generation # 45 Best fit:  23
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  23
Generation :  38
Solution found in  0.918004035949707 seconds

 Result grid:

392|185|647
786|942|531
431|267|895
---|---|---
854|916|732
973|651|284
815|423|769
---|---|---
172|536|498
569|872|143
428|319|657


----------------------------------------
Generation # 46 Best fit:  23
----------------------------------------

 Result made from

816|425|739
---|---|---
172|536|498
569|872|143
418|329|675


----------------------------------------
Generation # 67 Best fit:  21
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  21
Generation :  57
Solution found in  1.3544831275939941 seconds

 Result grid:

392|185|647
786|942|531
531|267|894
---|---|---
954|716|832
273|695|814
816|425|739
---|---|---
172|536|498
569|872|143
418|329|675


----------------------------------------
Generation # 68 Best fit:  21
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  21
Generation :  57
Solution found in  1.359762191772461 seconds

 Result grid:

392|185|647
786|942|531
531|267|894
---|---|---
954|716|832
273|695|814
816|425|739
---|---|---
172|536|498
569|872|143
418|329|675


---------------------------

273|895|614
816|423|759
---|---|---
172|536|498
569|873|124
841|329|675


----------------------------------------
Generation # 90 Best fit:  21
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  17
Generation :  86
Solution found in  1.8066391944885254 seconds

 Result grid:

396|185|247
786|942|531
431|257|896
---|---|---
954|716|382
273|895|614
816|423|759
---|---|---
172|536|498
569|873|124
841|329|675


----------------------------------------
Generation # 91 Best fit:  20
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  17
Generation :  86
Solution found in  1.8102891445159912 seconds

 Result grid:

396|185|247
786|942|531
431|257|896
---|---|---
954|716|382
273|895|614
816|423|759
---|---|---
172|536|498
569|873|124
841|329|675


--------------

...

876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 890 Best fit:  15
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  15.833197116851807 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 891 Best fit:  15
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  15.8594970703125 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
13

876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 915 Best fit:  16
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  16.259114980697632 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 916 Best fit:  15
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  16.26105809211731 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
1

Generation # 940 Best fit:  13
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  16.680989265441895 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 941 Best fit:  13
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  16.711435079574585 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 942 Best fit:  13
----------------------------------------

 Result mad

798|321|654


----------------------------------------
Generation # 965 Best fit:  17
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  17.100122213363647 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 966 Best fit:  17
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  17.102335929870605 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 967 Best fit:  1

Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  17.50657606124878 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 991 Best fit:  13
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  530
Solution found in  17.521543264389038 seconds

 Result grid:

389|165|247
876|942|531
241|537|896
---|---|---
957|416|382
623|857|914
814|296|735
---|---|---
132|758|469
564|879|123
798|321|654


----------------------------------------
Generation # 992 Best fit:  13
----------------------------------------

 Result made from base grid:  /Users/shreya/Desktop/Grid1.ss
Population size:  10
With Max Generations:  1000

 Best Fitness:  10
Generation :  53

***

 <h4> 2.1.2. (continued)  You will need to run experiments for the three Sudoku grids provided on the ELE page, for population sizes
10, 100, 1000, 10000. Each experiment (i.e., a specific combination of grid and population size) needs to
be ran 5 times (each one with a different random seed) and average performance across runs considered.
In total these amount to 3 x 4 x 5 = 60 runs. </h4>

***

<h3> Experiment Findings </h3>

Within the submission folder I have also provided a README file which outlines the details on how to run the solver (GA_Sudoku.py). The user is able to change the parameter values to run the experiments according to the population size required and grid to be used. I have kept the 'NUMBER_GENERATION' (the number of maximum generations) at 100,000 as it ensures that the experiments are fair and that it is not a limiting factor for the experiments.

For my results, I have kept compared two variables: the average time it takes the solver to find a solution and the population size. I have chosen to take the average time as I think the most effective algorithms should be able to find the correct solutions in the shortest period of time. 

As such, I have not chosen to take into account the number of generations as this would cause bias. This is because smaller populations would cover generations very quickly, hence not being an effective indicator of the performance level .

**I have attached my findings in the file called grid_info.pdf which contains the graph and tables of the three grids.**

***

<h4> Question 2.2: Analysis the Sudoko problem using Evolutionary algorithm. </h4>
<h4> 1. What population size was the best? </h4>

After completing the experiments with the three grids, I think that a population size of 100 was the most effective. It took much less time to reach solutions, and in the case of the 3rd grid, also managed to find the correct solution within 5 minutes.

<h4> 2. What do you think is the reason for your findings in question 2.2.1? </h4>

If the populations has been under 100, then the generations would be too volatile so it would be hard to detect if it has enetered local minima and hence would be difficult to decide if super mutation should be applied or not. On the other hand, populations greater than 100 would always have good individuals but would take far too long to reach solutions in time. Taking into account these considerations, a population size of 100 seems to be the optimum population.

<h4> 3. Which grid was the easiest and which the hardest to solve? </h4>

I found that the difficultly of the grids increased in order from Grid 1 to Grid 3. This is due to the fact the time spent on finding solutions for larger populations for each grid increased substantially every time. For Grid 3 especially, most of solutions were not found within the time limit of 5 minutes. Therefore it can be said Grid 1 was the easiest, and Grid 3 the hardest to solve.

<h4> 4. What do you think might be the reason for your findings in question 2.2.3? </h4>

The increase in difficultly may be due to the amount of pre-defined numbers (numbers which are set in the base grid) in each grid. When there are more pre-defined numbers in a grid, it is easier to solve. Fewer pre-defined numbers would mean there are more possibilites to traverse when crossing, mutating or generating individuals. In such a case where there are fewer pre-defined numbers, it would make finding the correct solution much more difficult and time consuming.

<h4> 5. What further experiments do you think it may be useful to do and why? </h4>

It may be benefitial to research the effect of modifiying the replacement rate, or truncation rate. I experimented with the truncation rate to find the number to use in my experiments, however it may be useful to have further experiments on this to find the ideal truncation rate. The ideal replacement rate is to find the optimal percentage of the best parents to replace the child population. It would also be benefital to have a look into the effect super mutations have in avoiding being stuck in a continous local minima. All of these experiments would be useful to uncover ideal parameters which would make the algorithm more effective.

***